In [2]:
import pandas as pd
import json

In [3]:
path_ = '..\\data\\data_processed_all_stations.json'

with open(path_, 'r') as file:
    document = json.load(file)
    
df = pd.DataFrame(document)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CD_OSCAR            712 non-null    object 
 1   DC_NOME             714 non-null    object 
 2   FL_CAPITAL          714 non-null    object 
 3   DT_FIM_OPERACAO     714 non-null    int64  
 4   CD_SITUACAO         714 non-null    object 
 5   TP_ESTACAO          714 non-null    object 
 6   VL_LATITUDE         714 non-null    float64
 7   CD_WSI              712 non-null    object 
 8   CD_DISTRITO         714 non-null    int64  
 9   VL_ALTITUDE         714 non-null    float64
 10  SG_ESTADO           714 non-null    object 
 11  CD_ESTACAO          714 non-null    object 
 12  VL_LONGITUDE        714 non-null    float64
 13  DT_INICIO_OPERACAO  714 non-null    object 
 14  REGIAO              714 non-null    object 
dtypes: float64(3), int64(2), object(10)
memory usage: 83.8+ K

In [5]:
df[['REGIAO','DC_NOME', 'SG_ESTADO', 'CD_ESTACAO']].head()


,REGIAO,DC_NOME,SG_ESTADO,CD_ESTACAO
0,NORDESTE,ABROLHOS,BA,A422
1,NORDESTE,ACARAU,CE,A360
2,SUDESTE,AFONSO CLAUDIO,ES,A657
3,CENTRO-OESTE,AGUA BOA,MT,A908
4,CENTRO-OESTE,AGUA CLARA,MS,A756


### Função reponsavel por verificar se sigla do estado está no mapa_regioes<br> Retorna o noma da regão

In [ ]:
def obter_regiao_pelo_estado(estado):
    mapa_regioes = {
        'NORTE': ['AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO'],
        'NORDESTE': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
        'CENTRO-OESTE': ['DF', 'GO', 'MT', 'MS'],
        'SUDESTE': ['ES', 'MG', 'RJ', 'SP'],
        'SUL': ['PR', 'RS', 'SC']
    }

    for regiao, estados in mapa_regioes.items():
        if estado in estados:
            return regiao

    return 'Região não encontrada'

### Cria uma nova coluna no dataframe estações

In [ ]:
def cria_coluna_regiao_no_datafrma(df):
    
    df['REGIAO'] = df['SG_ESTADO'].apply(lambda estado:  obter_regiao_pelo_estado(estado))


# SEÇÃO: CRIAR UMA LISTA DE ESTAÇÕES POR REGIÃO

In [41]:
df [['REGIAO','SG_ESTADO', 'DC_NOME', 'CD_ESTACAO']]  [df['REGIAO'] == 'SUL'  ].sort_values('CD_ESTACAO', ascending=False)

,REGIAO,SG_ESTADO,DC_NOME,CD_ESTACAO
398,SUL,RS,PORTO ALEGRE- BELEM NOVO,B807
142,SUL,PR,COLOMBO,B806
285,SUL,PR,LARANJEIRAS DO SUL,B804
99,SUL,PR,CAMPINA DA LAGOA,B803
521,SUL,RS,Santa Vitoria do Palmar - Barra do Chui,A899
...,...,...,...,...
642,SUL,RS,LAGOA VERMELHA,83916
665,SUL,RS,PASSO FUNDO,83914
616,SUL,RS,CRUZ ALTA,83912
631,SUL,PR,IRATI,83836


In [42]:
from datetime import datetime, timedelta
import time
import requests
import os
from dotenv import load_dotenv



class ConnectAPI:
    
    def __init__(self) -> None:
        
        self.url_base = 'https://apitempo.inmet.gov.br'
        self.token = self.get_token()
        


    # obter token
    def get_token(self):
        
        load_dotenv()
        
        token1 = os.getenv('TOKEN1')
        token2 = os.getenv('TOKEN2')
        
        if token1 and token2 is not None:
            self.token = True
        else:
            raise ValueError('Token não encontrado.')
        
        return f'{token1}={token2}'
    
    
    
    # Obter os dados de uma url
    def get_data(self, url):
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
        return data
        


    # obtem dados horarios baseados nos parametros
    def get_data_hour_of_station(self, data_inicial, data_final, station):

        url = f'{self.url_base}/token/estacao/{data_inicial}/{data_final}/{station}/{self.token}'
        print(url)
        
        data = self.get_data(url) 
        
        return data
        


    def incrementar_dia(self, start_day):
            
            # converte a data (string) para data(datetime)
            date_curent = datetime.strptime(start_day, '%Y-%m-%d').date()
            
            # recuperar a data de hoje
            data_hora_atual = datetime.now()
            data_de_hoje = data_hora_atual.date()
            
            # indentificar o interevalo entre data de hoje e a data inicial
            intervalo = (data_de_hoje - date_curent).days
                
            if intervalo >= 7:
                
                date_curent += timedelta(days=6)
            
            else:
                
                date_curent += timedelta(days=intervalo-1)
            
            new_start_day = date_curent.strftime('%Y-%m-%d')
            
            return new_start_day
        
        
    def get_data_hour_of_api(self, start_date, cod_estacao):
            
            dados_diarios = []  
            temp_date = datetime.strptime(start_date, '%Y-%m-%d').date()

            # recuperar a data de hoje
            data_hora_atual = datetime.now()
            data_de_hoje = data_hora_atual.date()
            
            while temp_date < data_de_hoje:
                
                end_date = self.incrementar_dia(start_date)
                
                dados_horarios_de_estacao = self.get_data_hour_of_station(start_date,end_date,cod_estacao)
                

                print({len(dados_horarios_de_estacao)})
                print(start_date, end_date)
                
                dados_diarios.extend(dados_horarios_de_estacao)
                
                temp_date = datetime.strptime(end_date, '%Y-%m-%d').date()
                temp_date = temp_date + timedelta(days=1)
                
                start_date = temp_date.strftime('%Y-%m-%d')
                time.sleep(30)
            
            return dados_diarios
        
    # função responsável por salvar dataframe em um arquivo json
    def salvar_dados_em_um_arquivo_json(self, df, nome_arquivo):
        
        path = self.obter_diretorio_data()
        
        df.to_json(f'{path}\{nome_arquivo}.json',orient='records')

In [43]:
series_de_estacoes = df [['CD_ESTACAO']]  [df['REGIAO'] == 'SUL'  ].sort_values('CD_ESTACAO', ascending=False)
lista_de_estacoes = list(series_de_estacoes['CD_ESTACAO'])
type(lista_de_estacoes)

list

In [44]:
len(lista_de_estacoes)

107

In [51]:
connect = ConnectAPI()

# PROCESSO: Extrai dados da API, baseado é uma data inicial e código de uma estação.
# --------------------------------------------------------------------------------
# 1 - Defini os parametros de entrada 'data_incial' e 'codigo_estacao'
# 2 - Obtem os dados e guarda em uma variavel dados
# 3 - Salva os dados em um arquivo json
#----------------------------------------------------------------------------------
start_date = '2024-01-01'
#cod_estacao = 'A101'   
#dados = connect.get_data_hour_of_api(start_date, cod_estacao)
#connect.salvar_dados_em_um_arquivo_json(dados, 'dados_brutos_regiao_sudeste')
#print(len(dados), type(dados))
temp_dados = []
for estacao in lista_de_estacoes:
    dados = connect.get_data_hour_of_api(start_date, estacao)
    temp_dados.extend(dados)
    print('tamanho: ', len(temp_dados))
    print('estação: ', estacao)


#while temp_dados:
#    temp_dados =  lista_de_estacoes.apply(lambda estacao: connect.get_data_hour_of_api(start_date, estacao))
#    dados.extend(temp_dados)
#    print(temp_dados)

#start_date = '2024-01-21'  
#dados = []
#dados.extend(lista_de_estacoes.apply(lambda estacao: connect.get_data_hour_of_api(start_date, estacao)))

https://apitempo.inmet.gov.br/token/estacao/2024-01-01/2024-01-07/B807/cFRCVjVEdWdEUXZ6cVZ6N2tmRFI1YVF1WUVmT0lTaDk=pTBV5DugDQvzqVz7kfDR5aQuYEfOISh9
{168}
2024-01-01 2024-01-07
https://apitempo.inmet.gov.br/token/estacao/2024-01-08/2024-01-14/B807/cFRCVjVEdWdEUXZ6cVZ6N2tmRFI1YVF1WUVmT0lTaDk=pTBV5DugDQvzqVz7kfDR5aQuYEfOISh9
{168}
2024-01-08 2024-01-14
https://apitempo.inmet.gov.br/token/estacao/2024-01-15/2024-01-21/B807/cFRCVjVEdWdEUXZ6cVZ6N2tmRFI1YVF1WUVmT0lTaDk=pTBV5DugDQvzqVz7kfDR5aQuYEfOISh9
{168}
2024-01-15 2024-01-21
https://apitempo.inmet.gov.br/token/estacao/2024-01-22/2024-01-28/B807/cFRCVjVEdWdEUXZ6cVZ6N2tmRFI1YVF1WUVmT0lTaDk=pTBV5DugDQvzqVz7kfDR5aQuYEfOISh9
{168}
2024-01-22 2024-01-28
https://apitempo.inmet.gov.br/token/estacao/2024-01-29/2024-01-30/B807/cFRCVjVEdWdEUXZ6cVZ6N2tmRFI1YVF1WUVmT0lTaDk=pTBV5DugDQvzqVz7kfDR5aQuYEfOISh9
{48}
2024-01-29 2024-01-30
tamanho:  720
estação:  B807
https://apitempo.inmet.gov.br/token/estacao/2024-01-01/2024-01-07/B806/cFRCVjVEdWdEUXZ6cVZ6N

In [47]:
len(temp_dados)

4320

In [48]:
df_test = pd.DataFrame(temp_dados)

In [49]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4320 entries, 0 to 4319
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DC_NOME       4320 non-null   object
 1   PRE_INS       4291 non-null   object
 2   TEM_SEN       3989 non-null   object
 3   VL_LATITUDE   4320 non-null   object
 4   PRE_MAX       4291 non-null   object
 5   UF            4320 non-null   object
 6   RAD_GLO       4291 non-null   object
 7   PTO_INS       4284 non-null   object
 8   TEM_MIN       4291 non-null   object
 9   VL_LONGITUDE  4320 non-null   object
 10  UMD_MIN       4284 non-null   object
 11  PTO_MAX       4283 non-null   object
 12  VEN_DIR       3756 non-null   object
 13  DT_MEDICAO    4320 non-null   object
 14  CHUVA         4291 non-null   object
 15  PRE_MIN       4291 non-null   object
 16  UMD_MAX       4284 non-null   object
 17  VEN_VEL       3996 non-null   object
 18  PTO_MIN       4283 non-null   object
 19  TEM_MA

In [50]:
df_test.memory_usage(deep=True).sum()

7140362

In [ ]:
connect.salvar_dados_em_um_arquivo_json(dados, 'dados_brutos_regiao_sudeste')